In [168]:
import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge, LinearRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.base import TransformerMixin
from sklearn.metrics import mean_squared_error, r2_score
import math
from scipy.stats import norm

In [169]:
#From Linear regression, we know that dropping Pool Area and Pool QC improve accuracy. Thus, used new raw data set that has those 
#features dropped
df = pd.read_csv('/Users/Alice Cheng/Documents/Data Science Final Project/AmesHousingBasedonSelectKBestandCorr.csv')
df.head()

,Overall Qual,Lot Area,Utilities,Neighborhood,Bldg Type,House Style,BsmtFin SF 1,Garage Area,TotRms AbvGrd,Gr Liv Area,Misc Feature,SalePrice
0,6,31770,AllPub,NAmes,1Fam,1Story,639.0,528.0,7,1656,NaN,215000
1,5,11622,AllPub,NAmes,1Fam,1Story,468.0,730.0,5,896,NaN,105000
2,6,14267,AllPub,NAmes,1Fam,1Story,923.0,312.0,6,1329,Gar2,172000
3,7,11160,AllPub,NAmes,1Fam,1Story,1065.0,522.0,8,2110,NaN,244000
4,5,13830,AllPub,Gilbert,1Fam,2Story,791.0,482.0,6,1629,NaN,189900


In [170]:
df_converted = pd.get_dummies(df)
df_converted.head()

,Overall Qual,Lot Area,BsmtFin SF 1,Garage Area,TotRms AbvGrd,Gr Liv Area,SalePrice,Utilities_AllPub,Utilities_NoSeWa,Utilities_NoSewr,...,House Style_2.5Fin,House Style_2.5Unf,House Style_2Story,House Style_SFoyer,House Style_SLvl,Misc Feature_Elev,Misc Feature_Gar2,Misc Feature_Othr,Misc Feature_Shed,Misc Feature_TenC
0,6,31770,639.0,528.0,7,1656,215000,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,11622,468.0,730.0,5,896,105000,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,14267,923.0,312.0,6,1329,172000,1,0,0,...,0,0,0,0,0,0,1,0,0,0
3,7,11160,1065.0,522.0,8,2110,244000,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,13830,791.0,482.0,6,1629,189900,1,0,0,...,0,0,1,0,0,0,0,0,0,0


In [171]:
df_filled=df_converted.fillna(0)

X=df_filled.drop(['SalePrice'],1)
y = df_filled['SalePrice']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def getDummy(df):
    df=pd.get_dummies(df)
    return df

# One Hot Encoding
X_train=getDummy(X_train)
X_test=getDummy(X_test)

# Perform Lasso to remove less performing variables
lasso = Lasso()

lasso.fit(X_train,y_train)

X_train=X_train.iloc[:,lasso.coef_!=0]
X_test=X_test.iloc[:,lasso.coef_!=0]
lasso = Lasso().fit(X_train,y_train)
print("Training set score: {:.2f}".format(lasso.score(X_train,y_train)))
print("Test set score: {:.2f}".format(lasso.score(X_test,y_test)))
print("Number of features used: {}".format(np.sum(lasso.coef_!=0)))

Training set score: 0.86
Test set score: 0.86
Number of features used: 50


C:\Anaconda\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [172]:
y_pred = lasso.predict(X_test)
# The mean square error
print("Mean squared error: %.2f"
      % mean_squared_error(y_test, y_pred))
#R^2 score
print('R^2 score: %.2f' % r2_score(y_test, y_pred))

Mean squared error: 1092050105.48
R^2 score: 0.86
